In [5]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
model = ChatOllama(
    model="gemma3:4b",
    temperature=0.7
)

In [6]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_info}"),
    ]
)

In [7]:
pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。"),
        ]
)

pros_branch_chain = (
    pros_template
    | model
    | StrOutputParser()
)

In [8]:
cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。"),
        ]
)

cons_branch_chain =(
    cons_template
    | model
    | StrOutputParser() 
    
)

In [9]:
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

In [10]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chain,
        cons = cons_branch_chain
    )
    | RunnableLambda(lambda x: combine_pros_cons(x['pros'],x['cons']))
)

In [11]:
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_info": restaurant_info})

print(result)


🍽️ 餐廳評論分析報告

✅ 優點分析：
好的，根據我分析，這家餐廳的優點如下：

*   **優雅的環境：** 餐廳位於台北市中心，環境設計融入了義大利的古典元素，營造出舒適、浪漫的氛圍。
*   **親切的服務：** 服務人員的親切是提升顧客滿意的關鍵。
*   **食物品質不錯：** 廚師在食材選擇和烹飪技巧方面有一定水準。

希望這些信息對您有幫助！


❌ 缺點分析：
根據我分析的結果，這家餐廳的缺點有以下幾個：

1. **價格偏高**：這是最顯著的缺點，超出許多消費者的預算。
2. **等待時間較長**：尤其在高峰期，顧客會因為等待時間過長而感到不滿。
3. **份量偏少**：對於價格高的餐廳來說，份量不足，讓顧客覺得物價不合理。
4. **缺乏對不同消費者的考慮**：目前餐廳主要針對約會或特殊場合的顧客，如果能考慮到單人用餐、家庭用餐等不同需求，將會更具吸引力。

總而言之，這家餐廳的缺點主要集中在**價值感**方面，需要進一步提升，才能提升顧客的滿意度。


📊 總結：這是一個平衡的餐廳評論分析

